<a href="https://colab.research.google.com/github/harjeet88/LLM_experiemnts/blob/main/RAG/RAG_sql_lite_website.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
import sqlite3
import torch
import numpy as np #Import numpy
from transformers import AutoTokenizer, AutoModel, pipeline

In [29]:
# Define the model and tokenizer
#model_name = "sentence-transformers/all-MiniLM-L6-v2"
#model_name = "sentence-transformers/multi-qa-mpnet-base-dot-v1"
#model_name = "meta-llama/Llama-2-7b-chat-hf"
model_name = "sentence-transformers/all-mpnet-base-v2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

In [20]:
# Setting up SQLite connection
conn = sqlite3.connect('rag_example.db')
cur = conn.cursor()

In [33]:
# Create table with pgvector extension
cur.execute("""
CREATE TABLE IF NOT EXISTS documents (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    content TEXT,
    embedding BLOB
);
""")
conn.commit()

In [34]:
# Function to compute embeddings
def compute_embedding(text):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.pooler_output[0].numpy()

In [35]:
# Inserting documents into the database
documents = [
    "The quick brown fox jumps over the lazy dog.",
    "The rain in Spain stays mainly in the plain.",
    "In a hole in the ground there lived a hobbit."
]

In [36]:
for doc in documents:
    embedding = compute_embedding(doc).tobytes()  # Convert numpy array to bytes
    cur.execute("INSERT INTO documents (content, embedding) VALUES (?, ?)", (doc, embedding))
conn.commit()

In [40]:
# Function to retrieve documents based on query
def retrieve_documents(query, top_k=3):
    query_embedding = compute_embedding(query)
    cur.execute("SELECT content, embedding FROM documents")
    rows = cur.fetchall()

    # Compute cosine similarity between query embedding and document embeddings
    similarities = []
    for content, embedding in rows:
        # Convert the byte embedding back to a numpy array, then a tensor
        doc_embedding = np.frombuffer(embedding, dtype=np.float32)
        doc_embedding = torch.tensor(doc_embedding).view(1, -1) #View is added to ensure correct dimensions
        similarity = torch.nn.functional.cosine_similarity(torch.tensor(query_embedding), doc_embedding)
        similarities.append((content, similarity.item()))

    # Sort by similarity and return top_k results
    similarities.sort(key=lambda x: x[1], reverse=True)
    return [content for content, _ in similarities[:top_k]]

In [44]:
# Function to generate a response
def generate_response(query):
    retrieved_docs = retrieve_documents(query)
    combined_text = " ".join(retrieved_docs)
    generator = pipeline("text-generation", model="gpt2")
    response = generator(f"Query: {query}\nDocuments: {combined_text}\nAnswer:", max_length=100)
    return response[0]['generated_text']

In [46]:
# Example query
query = "where does fox live"
response = generate_response(query)
print(response)

Device set to use cpu
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Query: where does fox live
Documents: The quick brown fox jumps over the lazy dog. In a hole in the ground there lived a hobbit. The rain in Spain stays mainly in the plain.
Answer: fox was born in the woods
Dogs living the woods
The real fox
So the real thing is not fox and just an idea. But when you come and get in your car in search of something to get your feet wet on, it goes the wrong way. It


In [ ]:

# Close the database connection
cur.close()
conn.close()